In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from itertools import combinations
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import ast
import random
import re

C:\Users\Rutuja Navale\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
train_df = pd.read_csv("orders_grouped_cleaned(1).csv")
train_df['parsed_items'] = train_df['item_name'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

segment_keys = ['CUSTOMER_TYPE', 'STORE_NUMBER', 'DAY_TYPE', 'ORDER_SUBCHANNEL_NAME']
segment_item_freq = {}
for key in segment_keys:
    segment_item_freq[key] = train_df.explode('parsed_items').groupby([key, 'parsed_items']).size().unstack(fill_value=0)

def get_segment_popularity(item, context_row):
    pop_features = []
    for key in segment_keys:
        val = context_row.get(key)
        if pd.notna(val) and val in segment_item_freq[key].index:
            pop = segment_item_freq[key].loc[val].get(item, 0)
        else:
            pop = 0
        pop_features.append(np.log1p(pop))
    return pop_features

def dropout_cart(cart, p=0.25):
    if len(cart) <= 2: return cart
    return [item for item in cart if random.random() > p]

order_df = train_df.groupby(['ORDER_ID', 'CUSTOMER_ID'])['parsed_items'].sum().reset_index()
order_df['parsed_items'] = order_df['parsed_items'].apply(lambda x: list(set(dropout_cart(x))))
order_df = order_df[order_df['parsed_items'].str.len() > 0]

item_sequences = order_df['parsed_items'].tolist()

In [3]:
w2v_model = Word2Vec(
    sentences=item_sequences,
    vector_size=50,
    window=3,
    min_count=2,
    workers=4,
    sg=1,
    seed=42
)

In [4]:
# for validation
test_df_feedback = pd.read_csv("test_data_question.csv")
test_df_feedback['cart_items'] = test_df_feedback[['item1', 'item2']].values.tolist()
test_df_feedback['actual_missing_item'] = test_df_feedback['item3']

fine_tune_data = [row['cart_items'] + [row['actual_missing_item']] for _, row in test_df_feedback.iterrows()]
if fine_tune_data:
    w2v_model.build_vocab(fine_tune_data, update=True)
    w2v_model.train(fine_tune_data, total_examples=len(fine_tune_data), epochs=3)

In [7]:
def get_cart_vector(cart):
    vectors = [w2v_model.wv[i] for i in cart if i in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_model.vector_size)

def cosine_similarity(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

item_counts = Counter([item for cart in item_sequences for item in cart])
item_popularity = pd.Series(item_counts)

cooccur = defaultdict(Counter)
for cart in item_sequences:
    for i, j in combinations(set(cart), 2):
        cooccur[i][j] += 1
        cooccur[j][i] += 1

def get_item_category(item):
    item = item.lower()
    if "fries" in item or "strips" in item or "sticks" in item:
        return "side"
    elif "drink" in item or "soda" in item or "beverage" in item or "20 oz" in item:
        return "drink"
    elif "combo" in item or "meal" in item or "feast" in item or "Pack" in item:
        return "combo"
    elif "wings" in item or "sub" in item or "crispy" in item:
        return "main"
    else:
        return "other"

def get_cart_category_vector(cart_items):
    return Counter([get_item_category(i) for i in cart_items])

def get_cart_entropy(cart):
    freq = Counter(cart)
    total = sum(freq.values())
    probs = [v / total for v in freq.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def get_pairwise_similarity(cart):
    pairs = list(combinations(cart, 2))
    sims = [cosine_similarity(w2v_model.wv[i], w2v_model.wv[j])
            for i, j in pairs if i in w2v_model.wv and j in w2v_model.wv]
    return np.mean(sims) if sims else 0

In [8]:
rerank_data = []
for _, row in test_df_feedback.iterrows():
    cart_items = row['cart_items']
    actual = row['actual_missing_item']
    cart_len = len(cart_items)
    cart_cat = get_cart_category_vector(cart_items)
    cart_entropy = get_cart_entropy(cart_items)
    pairwise_sim = get_pairwise_similarity(cart_items)
    category_count = len(set(get_item_category(i) for i in cart_items))

    frequent_items = item_popularity[item_popularity > 5].index.tolist()
    negatives = list(set(frequent_items) - set(cart_items) - {actual})
    hard_negatives = random.sample(negatives, min(40, len(negatives)))

    for label, item in [(1, actual)] + [(0, neg) for neg in hard_negatives]:
        if item not in w2v_model.wv:
            continue
        item_vec = w2v_model.wv[item]
        sims = [cosine_similarity(item_vec, w2v_model.wv[ci]) for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue
        seg_pops = get_segment_popularity(item, row)
        rerank_data.append({
            'sim_avg': np.mean(sims),
            'sim_max': np.max(sims),
            'sim_min': np.min(sims),
            'sim_std': np.std(sims),
            'sim_len': len(sims),
            'popularity': np.log1p(item_popularity.get(item, 0)),
            'cooccur_sum': sum([cooccur[item][ci] for ci in cart_items]),
            'mean_cooccur': sum([cooccur[item][ci] for ci in cart_items]) / cart_len,
            'cart_len': cart_len,
            'cat_mismatch': int(get_item_category(item) not in cart_cat),
            'cart_entropy': cart_entropy,
            'pairwise_sim': pairwise_sim,
            'category_count': category_count,
            'store_pop': seg_pops[1],
            'channel_pop': seg_pops[3],
            'daytype_pop': seg_pops[2],
            'guest_pop': seg_pops[0],
            'hit': label
        })

rerank_df = pd.DataFrame(rerank_data)
features = [col for col in rerank_df.columns if col != 'hit']
X = rerank_df[features]
y = rerank_df['hit']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
group_train = [41] * (len(X_train) // 41)

model = lgb.LGBMRanker(objective="lambdarank", n_estimators=500, learning_rate=0.05)
model.fit(X_train[:len(group_train)*41], y_train[:len(group_train)*41], group=group_train)
print("✅ LambdaRank model trained")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003166 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2390
[LightGBM] [Info] Number of data points in the train set: 32759, number of used features: 13
✅ LambdaRank model trained


In [9]:
def rerank_candidates(cart_items, context_row, top_k=3):
    cart_len = len(cart_items)
    cart_cat = get_cart_category_vector(cart_items)
    cart_entropy = get_cart_entropy(cart_items)
    pairwise_sim = get_pairwise_similarity(cart_items)
    category_count = len(set(get_item_category(i) for i in cart_items))
    candidate_set = set()
    for ci in cart_items:
        candidate_set.update(cooccur[ci].keys())
    candidate_set -= set(cart_items)
    candidate_scores = []
    for item in candidate_set:
        if item not in w2v_model.wv:
            continue
        item_vec = w2v_model.wv[item]
        sims = [cosine_similarity(item_vec, w2v_model.wv[ci]) for ci in cart_items if ci in w2v_model.wv]
        if not sims:
            continue
        seg_pops = get_segment_popularity(item, context_row)
        features_arr = np.array([[np.mean(sims), np.max(sims), np.min(sims), np.std(sims), len(sims),
                                  np.log1p(item_popularity.get(item, 0)),
                                  sum([cooccur[item][ci] for ci in cart_items]),
                                  sum([cooccur[item][ci] for ci in cart_items]) / cart_len,
                                  cart_len,
                                  int(get_item_category(item) not in cart_cat),
                                  cart_entropy,
                                  pairwise_sim,
                                  category_count,
                                  seg_pops[1], seg_pops[3], seg_pops[2], seg_pops[0]]])
        prob = model.predict(features_arr)[0]
        candidate_scores.append((item, prob))
    top_preds = sorted(candidate_scores, key=lambda x: x[1], reverse=True)[:top_k]
    pred_items = [item for item, _ in top_preds]
    if len(pred_items) < top_k:
        backup = [i for i in item_popularity.index if i not in cart_items]
        pred_items += backup[:top_k - len(pred_items)]
    return pred_items[:top_k]

def rerank_with_tta(cart_items, context_row, top_k=3, tta_rounds=5):
    vote_scores = defaultdict(float)
    for _ in range(tta_rounds):
        sampled_cart = random.sample(cart_items, len(cart_items))
        preds = rerank_candidates(sampled_cart, context_row, top_k=top_k)
        for rank, item in enumerate(preds):
            vote_scores[item] += 1 / (rank + 1)
    final_preds = sorted(vote_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return [item for item, _ in final_preds]

In [10]:
correct = 0
for _, row in test_df_feedback.iterrows():
    cart = row['cart_items']
    actual = row['actual_missing_item']
    preds = rerank_with_tta(cart, row)
    if actual in preds:
        correct += 1
print(f"🌟 Validation Recall@3: {correct / len(test_df_feedback) * 100:.2f}%")

🌟 Validation Recall@3: 78.80%


In [ ]:
hackathon_test_df = pd.read_csv("test_data_question.csv")  
submission_rows = []
for _, row in hackathon_test_df.iterrows():
    cart = [row['item1'], row['item2'], row['item3']]
    preds = rerank_with_tta(cart, row, top_k=3)
    submission_rows.append({
        "ORDER_ID": row["ORDER_ID"],
        "RECOMMENDATION_1": preds[0] if len(preds) > 0 else None,
        "RECOMMENDATION_2": preds[1] if len(preds) > 1 else None,
        "RECOMMENDATION_3": preds[2] if len(preds) > 2 else None
    })
submission_df = pd.DataFrame(submission_rows)
submission_df.to_excel("hackathon_submission.xlsx", index=False)
print("Submission file created: hackathon_submission.xlsx")